In [5]:
import os
from glob import glob
from pprint import pprint

### Set working directory to parent directory to import fields processing functions
os.chdir(os.path.pardir)

cwd = os.getcwd()
print(cwd)

py_files = glob('*.py')
print(py_files)

In [7]:
### Config parameters are stroed in the global_config.py, imported at the top of the fields_functions.py
### Import fields functions
from fields_functions import *

In [8]:
print(config)

{'prep_file_dir': 'C:/Users/jesse/Documents/grad school/masters research/code/fields_library/data/rasters/from_MSI/', 'prep_tile_id': 'TPT', 'prep_base_chunk': 'auto', 'prep_time_chunk': 'auto', 'prep_remove_overlap': False, 'prep_manual_subset': True, 'prep_x_start': 7500, 'prep_y_start': 7500, 'prep_step': 500, 'prep_cloud_coverage_thresh': 50, 'prep_load_cloud_mask': True, 'prep_apply_cloud_mask': True, 'prep_cloud_mask_thresh': 70, 'prep_clip_outliers': True, 'prep_clip_percentile': 1, 'prep_normalize_bands': True, 'preproc_out_dir': 'preproc_out_dir/', 'preproc_outfile_prefix': 'fields_preproc_demo_', 'preproc_sample_pct': 0.05, 'preproc_n_clusters': 15, 'preproc_cluster_tile': True, 'kmeans_n_clusters': 15, 'kmeans_model_out_dir': 'kmeans_model_dir/', 'kmeans_8var_clusters': True, 'kmeans_std_thresh': 0.2, 'kmeans_min_thresh': 0, 'kmeans_max_thresh': 0.3, 'kmeans_range_thresh': 0.7, 'kmeans_ndwi_thresh': 0.2, 'kmeans_mask_out_dir': 'mask_out_dir/', 'kmeans_from_full_tile_mask': F

In [10]:
import memory_profiler
%load_ext memory_profiler

In [11]:
%memit

print('memory test')

peak memory: 208.25 MiB, increment: 0.04 MiB
memory test


In [22]:

"""
-------------------------------------
Mask Processing functions
-------------------------------------
"""

### QUESTION : the data in each band gets normalized multiple times (red & nir in NDVI, green & nir in NDWI, and all bands in the edge map)
###             should this step happen once up front instead?

### NDVI
def ndvi_xr(input_ds):
    '''
    computes the Normalized Difference Vegetation Index
    '''
    np.seterr(divide='ignore', invalid='ignore')
    return ((input_ds.nir - input_ds.red)/(input_ds.nir + input_ds.red))

### UPDATE : the clip and normalize functions have been moved to the data prep step, so it shouldn't be necessary here
def ndvi_xr_norm(input_ds):
    np.seterr(divide='ignore', invalid='ignore')
    nir = normalize_ufunc(input_ds.nir)
    red = normalize_ufunc(input_ds.red)
    return ((nir - red)/(nir + red))

### EVI - // Enhanced Vegetation Index  (abbrv. EVI)
#// General formula: 2.5 * (NIR - RED) / ((NIR + 6*RED - 7.5*BLUE) + 1)
#// URL https://www.indexdatabase.de/db/si-single.php?sensor_id=96&rsindex_id=16
def evi_xr(input_ds):
    '''
    computes the Enhanced Vegetation Index
    '''
    np.seterr(divide='ignore', invalid='ignore')    
    input_ds = clip_nan_ufunc(input_ds, 1)
    input_ds = normalize_ufunc(input_ds)
    EVI = 2.5 * (input_ds.nir - input_ds.red) / ((input_ds.nir + 6.0 * input_ds.red - 7.5 * input_ds.blue) + 1.0)
    return EVI

### GCVI - green chlorophyll vegetation index
### https://digitalcommons.unl.edu/cgi/viewcontent.cgi?article=1278&context=natrespapers
### (NIR - green) / 1
### This isn't used
def gcvi_xr(input_ds):
    np.seterr(divide='ignore', invalid='ignore')
    return ((input_ds.nir - input_ds.green) - 1)

### NDWI
def ndwi_xr(input_ds):
    """
    computes the Normalized Difference Water Index
    """
    np.seterr(divide='ignore', invalid='ignore')
    return ((input_ds.green - input_ds.nir)/(input_ds.green + input_ds.nir))

### UPDATE : the clip and normalize functions have been moved to the data prep step, so it shouldn't be necessary here
def ndwi_xr_norm(input_ds):
    """
    computes the Normalized Difference Water Index
    """
    np.seterr(divide='ignore', invalid='ignore')
    green = normalize_ufunc(input_ds.green)
    nir = normalize_ufunc(input_ds.nir)
    return ((green - nir)/(green + nir))

### Find threshold value for NDWI - 0.5 to start
    ### UPDATE : switched from ndwi_xr_norm() to ndwi_xr() because the normalization takes place in the data prep now
def ndwi_mask_func(input_ds, ndwi_thresh = 0.5):
    ### Take the mean NDWI value across the full time stack, normalized  
    return xr.where(ndwi_xr_norm(input_ds).mean(dim='time', skipna=True) > ndwi_thresh, 1, 0).compute()
#    return xr.where(ndwi_xr(input_ds).mean(dim='time', skipna=True) > ndwi_thresh, 1, 0)

### EDGES
# Sobel edges function from skimage
def sobel_edges(input_array):
    return sobel(input_array)


### Prior version of the edges algorithm implementation did not provide accurate outputs, emphasizing noise in the time stack
def compute_edges(ds_time_stack, edge_algo = sobel_edges, chunk_size='auto', percentile=0.1):
    for i in np.arange(0,len(ds_time_stack.coords['time']),1):
        edges = xr.ufuncs.fabs(ds_time_stack.isel(time=i).fillna(0).map(edge_algo)).chunk({'x':chunk_size,'y':chunk_size})
        if i == 0:
            edges_sum = (edges['red']+edges['green']+edges['blue']+edges['nir'])
        if i > 0:
            edges_sum = edges_sum + (edges['red']+edges['green']+edges['blue']+edges['nir'])
        
#    edges_sum = normalize_ufunc(clip_nan_ufunc(edges_sum/(4*len(ds_time_stack.coords['time'])), percentile))
    edges_sum = edges_sum/(4*len(ds_time_stack.coords['time']))
    edges_sum = clip_nan_ufunc(edges_sum, percentile)
    edges_sum = normalize_ufunc(edges_sum)

    return edges_sum

"""
These functions are from the original thresholding approach
The cumulative edge algorithm here is slower than the one above (I think)
The edge finding approach has a lot of room for improvement.
"""
# Passes each band in each timestep of a xr dataset through the sobel edge filter in parallel, lazily evaluated
# Concatenates the sum of each timestep's edges into a xr dataarray in teh same shape as the input
### This could be broken up/made more modular for cases where there is only a single timestep, for instance
def edges_to_xr_data_array(input_ds, ndvi = False):
    # Define some lists to get populated in the loop
    edges_list = []
    edges_time_list = []
    
    bands = ['red','green','blue','nir']
    
    # List of bands to loop over
    if ndvi:
        bands = ['red','green','blue','nir','ndvi']
    
    # Loop through each time step to populate a list of summed edges for each band in each time step
    for i in np.arange(0,len(input_ds.coords['time']),1):
#        if input_ds.isnull:
#            continue
        
        # Loop through each band to compute edges and append resulting dataarray to a list
        for band in bands:
            # define coords for 
            coords = [input_ds[band].coords['y'],input_ds[band].coords['x']]
            
            # Run the sobel edges algorithm on overlapping chunks to cover seams in chunks because it is a focal operation
            # The input band (ds[band]) gets normalized so values range from 0 to 1 and then clipped to remove 
            # outliers at the highest and lowest 2% of values
            # prep edges input by normalizing the band
            edges_input_norm = normalize_ufunc(clip_nan_ufunc(input_ds[band].isel(time=i),2))
            # lazily pass array through the sobel_edges() function via the dask.map_overlap() function
            band_egdes_output = edges_input_norm.data.map_overlap(sobel_edges, depth=1)
            
            # Convert the edges array to a dataarray using the x and y coordinates of the input array
            band_egdes_output_da = xr.DataArray(band_egdes_output,
                                                dims=('y','x'),
                                                coords=coords)
            
            # Add the time step dimension to the edges array that matches the time coordinate for the time step being processed
            band_egdes_output_da['time'] = input_ds['time'][i]
            
            # append the edges to a list of DataArrays, one for each band for each time step
            edges_list.append(band_egdes_output_da)
        
        edges_list_da = xr.concat(edges_list, dim='time')
        # For each time step, sum the edges for each band
#        edges_sum = edges_list_da.sum(dim='time', skipna=True)
        edges_sum = edges_list_da.mean(dim='time', skipna=True)
        # Give this new dataarray a logical name
        edges_sum.name = 'edges_sum'
        # Append the summed edge map of each time step to a list, to be merged along the 'time' dimension
        edges_time_list.append(edges_sum)
    # concat the summed edges for each time step into an array that matches the shape (x, y, time) of the time_stack_ds
    edges_da = xr.concat(edges_time_list, dim='time')

    return edges_da


### build mask
    
def mask_ndvi_max_and_range(monthly_avg_ndvi_ds, max_thresh = 0.3, range_thresh = 0.3):
    ### Maximum monthly average NDVI
    ndvi_monthly_max = monthly_avg_ndvi_ds.max(dim='time', skipna = True)
    ### NDVI range from max monthly NDVI to mean NDVI for May
    ndvi_range_max_t0 = ndvi_monthly_max - monthly_avg_ndvi_ds.isel(time=0)
    ndvi_range_mask = xr.where(ndvi_range_max_t0 < range_thresh, 1, 0)
    ### Max monthly average ndvi
    ndvi_max_mask = xr.where(ndvi_monthly_max < max_thresh, 1, 0)
    ### Combine masks
    return xr.ufuncs.logical_or(ndvi_range_mask, ndvi_max_mask)

# This function is in the mask.py file
def create_combined_ndvi_edges(ndvi, edges, ndvi_weight=1, edges_weight=1):
    return (ndvi_weight*ndvi) + (edges_weight*edges)

def create_mask_bool_array(ndvi_edge_combo, binary_threshold):
    return xr.where(normalize_ufunc(ndvi_edge_combo) < binary_threshold, 1, 0)
#    return xr.where(normalize_ufunc(ndvi_edge_combo) < binary_threshold, False, True)

def build_mask(ndvi_range, cumulative_edges, ndvi_weight, edges_weight):
    # inputs get normalized so that each range from 0 to 1
    combined_mask_inputs = create_combined_ndvi_edges(ndvi = 1 - normalize_ufunc(ndvi_range), 
                                      edges = normalize_ufunc(cumulative_edges), 
                                      ndvi_weight = ndvi_weight, 
                                      edges_weight = edges_weight)

    # Compute mask
    return combined_mask_inputs

# This will get passed to dask arrays within the dataset through dask.map_overlap() for parallelization
def fill_holes(input_array):
    return ndi.binary_fill_holes(input_array)

# This will get passed to dask arrays within the dataset through dask.map_overlap() for parallelization
### Original preliminary run used a size = (2,2)
def min_filter(input_array, size = (2, 2)):
    return ndi.minimum_filter(input_array, size=size)


def mask_processing(input_ds, ndwi_thresh = 0.5, ndvi_max_thresh = 0.3, ndvi_range_thresh = 0.3, edges_thresh = 0.3):
    ### NDWI mask: mean NDWI across all time steps, masked for values greater than ndwi_thresh
    ndwi_mask = ndwi_mask_func(input_ds, ndwi_thresh = ndwi_thresh)
    
    ### NDVI Masking
    # Compute monthly mean composites
    monthly_avg = input_ds.resample(time='1MS').mean(skipna = True)
    # Compute NDVI on the monthly composite iamges
    monthly_avg_ndvi = ndvi_xr_norm(monthly_avg)
    # Compute NDVI mask: Max monthly mean NDVI > max_thresh, and Max Monthly NDVI - May monthly mean NDVI 
    combined_ndvi_mask = mask_ndvi_max_and_range(monthly_avg_ndvi, 
                                                 max_thresh = ndvi_max_thresh, 
                                                 range_thresh = ndvi_range_thresh)
    
    ### Edges
    # Mean edges for full data stack
    edges_mean = normalize_ufunc(clip_nan_ufunc(edges_to_xr_data_array(input_ds).mean(dim='time', skipna = True), 2))
    # Mask edges
    edges_mask = xr.where(edges_mean > edges_thresh, 1, 0)
    
    
    ### Combined mask
    combined_mask = xr.ufuncs.logical_or(xr.ufuncs.logical_or(ndwi_mask, combined_ndvi_mask), edges_mask)
    # Invert mask
    mask_invert = xr.where(combined_mask == 1, 0, 1)
    
    ### Fill holes in the mask so that there aren't stray pixels
    mask_holes_filled = mask_invert.data.map_overlap(fill_holes, depth=1)
    
    ### Set minimum filter to enforce minimum height/width of background. 
    ### This eliminates small isolated areas and expands road areas.
    mask_minimum_filt = mask_holes_filled.map_overlap(min_filter, depth = 1)
    
    ### Set mask to ds_time_stack array
    input_ds['mask'] = xr.DataArray(mask_minimum_filt, 
                                     dims=('y','x'),
                                     coords = [input_ds['red'].coords['y'],
                                               input_ds['red'].coords['x']])
    
    return input_ds

In [12]:
%%time

### Loaing data from a directory of zipped Sentinel-2 files

ds_time_stack = prep_data()

Cloud Coverage Threshold: 50
FAILED cloud coverage: 2019 05 03 with  99.009547 pct | nodata pixel pct: 0.126635
FAILED cloud coverage: 2019 05 06 with  72.591515 pct | nodata pixel pct: 5.957223
passed cloud coverage: 2019 05 13 with 34.97001 pct | nodata pixel pct: 0.134791
FAILED cloud coverage: 2019 05 16 with  72.077132 pct | nodata pixel pct: 5.796779
FAILED cloud coverage: 2019 05 23 with  96.121381 pct | nodata pixel pct: 0.149615
FAILED cloud coverage: 2019 05 26 with  73.563154 pct | nodata pixel pct: 5.719112
passed cloud coverage: 2019 06 02 with 21.420233 pct | nodata pixel pct: 0.193563
passed cloud coverage: 2019 06 05 with 3.328879 pct | nodata pixel pct: 5.51809
passed cloud coverage: 2019 06 12 with 3.1731840000000004 pct | nodata pixel pct: 0.164777
FAILED cloud coverage: 2019 06 15 with  91.997779 pct | nodata pixel pct: 5.732784
FAILED cloud coverage: 2019 06 22 with  95.609655 pct | nodata pixel pct: 0.121118
passed cloud coverage: 2019 06 25 with 31.59953999999999

In [13]:
%%time
%memit

## Example code for running Thresholding approach
## These functions are not integrated into the global config, so variables have to be defined before the function call

### Set file paths for read and write
file_out_str = '_code_demo_threshold'
out_dir = config['shp_out_dir']
tile_id = config['prep_tile_id']
raster_dir = config['prep_file_dir']
x_start = config['prep_x_start']
y_start = config['prep_y_start']
step = config['prep_step']
base_chunk = 'auto'

## Data read parameters
cloud_thresh = 20

## Crop maask parameters
ndwi_thresh = 0.5 
ndvi_max_thresh = 0.5
ndvi_range_thresh = 0.5 
edges_thresh = 0.3

# set rgb date string from the dict
rgb_date_str = rgb_date_dict[tile_id]
rgb_gaussian_sigma = config['seg_rgb_gaussian_sigma']
fz_scale = config['seg_fz_scale']
fz_sigma = config['seg_fz_sigma']
fz_min_size = config['seg_fz_min_size']

peak memory: 237.80 MiB, increment: 0.00 MiB
Wall time: 5.63 s


In [17]:
%%time
# %memit
ndwi_mask = ndwi_mask_func(ds_time_stack, ndwi_thresh = ndwi_thresh)

Wall time: 15 ms


In [23]:
%%time
ndwi_mask_compute = ndwi_mask_func(ds_time_stack, ndwi_thresh = ndwi_thresh)

Wall time: 32.5 s


In [19]:
%memit

peak memory: 930.60 MiB, increment: -0.17 MiB
